In [0]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

# spark = SparkSession.builder.appName("silver_promos").getOrCreate()

# --- 1. Definición de Rutas ---
BRONZE_PROMOS = 'damm_silver_des.gest_com_horeca.ca_crm_promociones'
SILVER_PROMOS = 'damm_silver_des.pricing_unho.ca_crm_promos_test'
SILVER_PROMOS_ESTABS = 'damm_silver_des.gest_com_horeca.ca_crm_establecimiento_objetivo'


spark = SparkSession.builder \
    .appName("YourApp") \
    .config("spark.network.timeout", "600s") \
    .config("spark.executor.heartbeatInterval", "60s") \
    .config("spark.sql.broadcastTimeout", "600") \
    .getOrCreate()


# --- 2. Lectura de Datos ---
df_promos_raw = spark.table(BRONZE_PROMOS)
df_establecimientos = (
    spark.table(SILVER_PROMOS_ESTABS)
    .select("Establecimiento", "PromoID")
    .withColumn("PromoID", F.regexp_replace(F.col("PromoID"), "^0+", ""))
)
print("Lectura de tablas de origen completada exitosamente.")



# --- 3. Transformación Principal ---
df_silver = (
    df_promos_raw
    .withColumn("FechaInicio", F.to_date(F.col("FechaIniDescuento"), "dd.MM.yyyy"))
    .withColumn("FechaFin", F.to_date(F.col("FechaFinDescuento"), "dd.MM.yyyy"))
    .withColumn("CantidadN", F.col("CantidadN").cast("int"))
    .withColumn("CantidadM", F.col("CantidadM").cast("int"))
    .groupBy("PromoID", "CodigoClaseDePromocion", "FechaInicio", "FechaFin", "PromoIDDesc1")
    .agg(
        F.collect_set(
            F.when(F.col("PromoN_M") == 'N',
                   F.struct(F.col("CodigoMaterial").alias("Material"),
                            F.col("CantidadN").alias("Cantidad")))
        ).alias("Condicion"),
        F.collect_set(
            F.when(F.col("PromoN_M") == 'M',
                   F.struct(F.col("CodigoMaterial").alias("Material"),
                            F.col("CantidadM").alias("Cantidad")))
        ).alias("Bonificacion"),
        F.when(
            F.max(F.when(F.col("PromoN_M") == 'M', F.col("CantidadM"))) == 0,
            F.lit("A_ELEGIR")
        ).otherwise(
            F.lit("TODOS")
        ).alias("TipoBonificacion")
    )
    .filter((F.size(F.col("Condicion")) > 0) & (F.size(F.col("Bonificacion")) > 0))
    .filter(F.col("PromoID").isNotNull())
    .select(
        F.col("PromoID").alias("CondicionID"),
        F.col("CodigoClaseDePromocion").alias("ClaseCondicion"),
        "FechaInicio",
        "FechaFin",
        "Condicion",
        "Bonificacion",
        "TipoBonificacion"
    )
)

# --- 4. Join con Establecimientos ---
df_final = (
    df_establecimientos
    .join(
        df_silver,
        on=df_establecimientos.PromoID == df_silver.CondicionID,
        how="left"
    )
    .select(
        "Establecimiento",
        "CondicionID",
        "FechaInicio",
        "FechaFin",
        "ClaseCondicion",
        "Condicion",
        "Bonificacion",
        "TipoBonificacion"
    )
    .filter(F.col("CondicionID").isNotNull())
    .distinct()
)

In [0]:

# --- 5. Escritura en la Tabla Silver ---
print(f"Escribiendo los datos procesados en la tabla: {SILVER_PROMOS}")
(df_final.write
    .mode("overwrite")
    .format("delta")
    .option("overwriteSchema", "true")
    .saveAsTable(SILVER_PROMOS)
)
print(f"¡Proceso completado! La tabla {SILVER_PROMOS} ha sido creada/actualizada correctamente.")
